In [1]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("../../../")
os.getcwd()

'/root/.cache/bazel/_bazel_root/4ba0b84eadbbe44ac95feee54d73b56c/execroot/com_nvidia_isaac_sdk/bazel-out/k8-opt/bin/packages/universal_robots/apps/simple_joint_control.runfiles/com_nvidia_isaac_sdk'

In [2]:
from IPython.display import display
import json
import numpy as np
import time

from packages.pyalice import Application, Codelet, Message
from packages.pyalice.gui.composite_widget import CompositeWidget

np.set_printoptions(precision=3)

In [4]:
# A Python codelet for joint control through widget
class JointPositionControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "state")
        self.tx = self.isaac_proto_tx("CompositeProto", "command")
        
        joints = self.config.joints
        limits = self.config.limits
        measure = self.config.measure
        self.stop_control = 3
        self._widget = CompositeWidget(joints, measure, limits)
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return
        display(self._widget.panel)

        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return
        self._widget.composite = state_msg

        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()
            
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

# Obtain the IP from UR Console and update this
IP = "192.168.56.1"
# Robot generation that you are going to control "e-series" or "cb3"
GENERATION = "e-series"

['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']


UR10 Hardware Joint Control
======

In [5]:
app = Application(name="simple_joint_control_ur_hardware", modules=["sight"])

if GENERATION == "e-series":    
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
elif GENERATION == "cb3":
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")
else:
    Exception("Unknown robot generation")

# Load components for configuration
ur_interface = app.nodes["ur.subgraph"]["interface"]
ur_controller = app.nodes["ur.controller"]["ScaledMultiJointController"]
ur_driver = app.nodes["ur.universal_robots"]["UniversalRobots"]

# configs
ur_controller.config.control_mode = "joint speed"
ur_driver.config.control_mode = "joint speed"
ur_driver.config.robot_ip = IP
ur_driver.config.headless_mode = False

# add pycodelet for joint position control
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = 'position'

# edges
app.connect(joint_commander, "command", ur_interface, "joint_target")
app.connect(ur_interface, "arm_state", joint_commander, "state")

# Enable sight
widget = app.add("sight").add(app.registry.isaac.sight.SightWidget, "simple_joint_control")
widget.config.type = "plot"
widget.config.channels = [
  {
    "name": "ur.universal_robots/UniversalRobots/shoulder_pan_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/shoulder_lift_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/elbow_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_1_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_2_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_3_joint"
  }

]

# start application
app.start()


2021-05-04 12:31:28,784 DEBUG Binding PyCodelet command_generator/PyCodelet
2021-05-04 12:31:28,787 DEBUG Launching isaac core
2021-05-04 12:31:28,789 DEBUG Launching pycodelet threads
2021-05-04 12:31:28,795 DEBUG Launching command_generator/PyCodelet


In [7]:
app.stop()

2021-04-29 12:23:34,917 DEBUG Stopped command_generator/PyCodelet
2021-04-29 12:23:34,918 DEBUG Python Codelets All stopped...


UR10 Digital IO Control
=====

In [6]:
app = Application(name="io_control_ur_hardware", modules=["sight"])

if GENERATION == "e-series":    
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
elif GENERATION == "cb3":
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")
else:
    Exception("Unknown robot generation")

# Load components for configuration
ur_interface = app.nodes["ur.subgraph"]["interface"]
ur_driver = app.nodes["ur.universal_robots"]["UniversalRobots"]

io_names = ur_driver.config.tool_digital_out_names
ur_driver.config.robot_ip = IP

# add pycodelet for digitalIO control
widget_node = app.add("command_generator")
io_commander = widget_node.add(JointPositionControl)
io_commander.config.joints = io_names
io_commander.config.limits = [[0, 1]] * len(io_names)
io_commander.config.measure = 'none'

# Edges
app.connect(io_commander, "command", ur_interface, "io_command")
app.connect(ur_interface, "io_state", io_commander, "state")

# Enable sight
widget = app.add("sight").add(app.registry.isaac.sight.SightWidget, "shuffle_box_hardware")
widget.config.type = "plot"
widget.config.channels = [
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_out_0"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_out_1"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_in_0"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_in_1"
  }
]

app.start()

2021-05-04 12:53:50,384 DEBUG Binding PyCodelet command_generator/PyCodelet
2021-05-04 12:53:50,387 DEBUG Launching isaac core
2021-05-04 12:53:50,388 DEBUG Launching pycodelet threads
2021-05-04 12:53:50,389 DEBUG Launching command_generator/PyCodelet


In [7]:
app.stop()

2021-05-04 12:55:22,145 DEBUG Stopped command_generator/PyCodelet
2021-05-04 12:55:22,146 DEBUG Python Codelets All stopped...
